In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test with Jamie

In [7]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_new_training_week_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training schedule shows a preference for workouts on Wednesdays and long runs on Saturdays, aligning well with your goals. You typically include rest or lighter running days on Sundays and incorporate moderate pacing throughout the week. To enhance marathon preparation, incorporating specific speed work on Wednesdays and ensuring a consistent long run on Saturdays will be beneficial., weekly_mileage_target=For Week 35, let's aim to slightly increase your mileage while still maintaining a conservative approach to ensure you're building safely. Target a total weekly mileage of around 46-47 miles, with your long run being approximately 16.5-17 miles. This progression aligns closely with your recent training pattern, allowing for a small but manageable increase that should keep you on t

In [8]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
athlete_id=98390356 token still valid until 2024-08-28 20:18:36+00:00
TrainingWeekWithPlanning(planning=The target for the week is to run a total of 46.0 miles. The client has already run 10.04 miles, leaving 35.96 miles to cover. The current plan adds up to 34.0 miles, which means we need to add 1.96 miles to the plan. There are five sessions left: a speed workout on Wednesday, a rest day on Thursday, an easy run on Friday, a long run on Saturday, and an easy run on Sunday. The rest day on Thursday does not contribute mileage. To distribute the extra mileage, we can slightly extend the length of the Friday easy run and the Sunday easy run, as the client may prefer not to add to the speed workout or long run. Currently, the Friday run is planned at 5.0 miles and the Sunday run at 4.0 miles. We will add 1.0 mile to the Friday run and 0.96 miles to the Sunday run. The adjusted plan is as follows: Wednesday speed workou

# Training week accuracy

In [13]:



path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in files:
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)

Loading 2024-08-28_16-47-43.json


In [23]:
# [result for result in results if result['error'] > 5]

for result in results:
    if result['error'] > 5:
        pprint(result)
        print("---")

{'error': 6.0,
 'generated_weekly_mileage': 34.0,
 'planning': 'To plan a balanced training week increasing to 27-28 miles with '
             'a 17-18 mile long run, I will distribute the remaining mileage '
             'across the week with one speed workout and efforts on most days '
             'for balanced training and recovery. Long run will be scheduled '
             "on Saturday based on client's history, speed work on Wednesday, "
             'and a rest day on Friday. Total planned mileage target is 27-28 '
             'miles. Adding an easy run on Mon with 4 miles, Tuesday moderate '
             'run of 5 miles, speed workout of 4 miles, rest on Friday (0m), '
             'Long run Saturday of 17 miles, and short recovery of 2 miles on '
             'Sunday. This totals 4 + 5 + 4 + 4 + 17 + 2 = 36 miles, adjust '
             'Thursday’s easy run to 2 miles makes it 27 miles total, within '
             'the 27-28 mileage target.',
 'recommended_weekly_mileage': '27

In [ ]:
""